In [1]:
#!/usr/bin/env python
# coding: utf-8

# # Pronóstico de Casos de Dengue por Barrio usando GRU
# ## Red Neuronal Recurrente para Series Temporales Multivariadas

# ### 1. Configuración e Importación de Librerías

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
import optuna
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurar dispositivo de cómputo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo de cómputo: {device}")

# Establecer semilla para reproducibilidad
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# ### 2. Carga y Preparación de Datos

# Cargar datos desde el archivo Parquet
df = pd.read_parquet('../../Datos/df_train.parquet')
print(f"Dimensiones del DataFrame: {df.shape}")
print("\nPrimeras filas:")
print(df.head())
print("\nInformación del DataFrame:")
print(df.info())

# Crear columna de fecha usando formato de semanas ISO
df['fecha'] = pd.to_datetime(df['anio'].astype(str) + df['semana'].astype(str) + '1', format='%G%V%u')

# Establecer fecha como índice y ordenar
df = df.set_index('fecha').sort_index()
print(f"\nRango temporal: {df.index.min()} - {df.index.max()}")

# ### 3. Preprocesamiento e Ingeniería de Características

# División de datos según el año
train_data = df[df['anio'] < 2021]
val_data = df[df['anio'] == 2021]
print(f"\nTamaño conjunto entrenamiento: {len(train_data)}")
print(f"Tamaño conjunto validación: {len(val_data)}")

# Identificar columnas numéricas y categóricas
categorical_cols = ['id_bar', 'ESTRATO']
numerical_cols = [col for col in df.columns if col not in categorical_cols + ['anio', 'semana']]
target_col = 'dengue'

print(f"\nColumnas categóricas: {categorical_cols}")
print(f"Columnas numéricas: {numerical_cols}")
print(f"Variable objetivo: {target_col}")

# Codificar variables categóricas para embeddings
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col + '_encoded'] = le.fit_transform(df[col])
    label_encoders[col] = le
    print(f"\nCardinalidad de {col}: {len(le.classes_)}")

# Actualizar división con columnas codificadas
train_data = df[df['anio'] < 2021]
val_data = df[df['anio'] == 2021]

# Escalar variables numéricas
scaler = StandardScaler()
scaler.fit(train_data[numerical_cols])

# Aplicar escalado
train_scaled = train_data.copy()
val_scaled = val_data.copy()
train_scaled[numerical_cols] = scaler.transform(train_data[numerical_cols])
val_scaled[numerical_cols] = scaler.transform(val_data[numerical_cols])

# ### Función para crear secuencias

def create_sequences(data, window_size, target_col, feature_cols, categorical_cols_encoded):
    """
    Crea secuencias de ventanas deslizantes para el modelo GRU.
    
    Args:
        data: DataFrame con los datos
        window_size: Tamaño de la ventana de entrada
        target_col: Nombre de la columna objetivo
        feature_cols: Lista de columnas de características numéricas
        categorical_cols_encoded: Lista de columnas categóricas codificadas
    
    Returns:
        sequences_num: Array con las secuencias numéricas
        sequences_cat: Array con las secuencias categóricas
        targets: Array con los valores objetivo
        barrio_ids: Array con los IDs de barrio para cada secuencia
    """
    sequences_num = []
    sequences_cat = []
    targets = []
    barrio_ids = []
    
    # Agrupar por barrio para mantener continuidad temporal
    for barrio in data['id_bar'].unique():
        barrio_data = data[data['id_bar'] == barrio].sort_index()
        
        if len(barrio_data) <= window_size:
            continue
            
        for i in range(len(barrio_data) - window_size):
            # Secuencia de características numéricas
            seq_num = barrio_data[feature_cols].iloc[i:i+window_size].values
            # Secuencia de características categóricas
            seq_cat = barrio_data[categorical_cols_encoded].iloc[i:i+window_size].values
            # Valor objetivo
            target = barrio_data[target_col].iloc[i+window_size]
            
            sequences_num.append(seq_num)
            sequences_cat.append(seq_cat)
            targets.append(target)
            barrio_ids.append(barrio)
    
    return (np.array(sequences_num, dtype=np.float32), 
            np.array(sequences_cat, dtype=np.int64),
            np.array(targets, dtype=np.float32),
            np.array(barrio_ids))

# ### 4. Definición del Modelo GRU con PyTorch

class DengueDataset(Dataset):
    """Dataset personalizado para las secuencias de dengue"""
    def __init__(self, sequences_num, sequences_cat, targets):
        self.sequences_num = sequences_num
        self.sequences_cat = sequences_cat
        self.targets = targets
    
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, idx):
        return (torch.FloatTensor(self.sequences_num[idx]),
                torch.LongTensor(self.sequences_cat[idx]),
                torch.FloatTensor([self.targets[idx]]))

class GRUModel(nn.Module):
    """
    Modelo GRU con embeddings para variables categóricas
    """
    def __init__(self, num_features, embedding_dims, hidden_size, num_layers, dropout_rate):
        super(GRUModel, self).__init__()
        
        # Embeddings para variables categóricas
        self.embeddings = nn.ModuleList()
        total_embedding_size = 0
        
        for card, dim in embedding_dims:
            self.embeddings.append(nn.Embedding(card, dim))
            total_embedding_size += dim
        
        # Tamaño de entrada para GRU
        input_size = num_features + total_embedding_size
        
        # Capas GRU
        self.gru = nn.GRU(input_size, hidden_size, num_layers, 
                         batch_first=True, dropout=dropout_rate if num_layers > 1 else 0)
        
        # Dropout
        self.dropout = nn.Dropout(dropout_rate)
        
        # Capa de salida
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x_num, x_cat):
        batch_size, seq_len, _ = x_num.shape
        
        # Procesar embeddings categóricos
        embeddings_list = []
        for i, embedding in enumerate(self.embeddings):
            # x_cat[:, :, i] selecciona la i-ésima variable categórica para todas las secuencias
            cat_embedded = embedding(x_cat[:, :, i])
            embeddings_list.append(cat_embedded)
        
        # Concatenar embeddings
        cat_embedded = torch.cat(embeddings_list, dim=-1)
        
        # Concatenar características numéricas y categóricas
        x = torch.cat([x_num, cat_embedded], dim=-1)
        
        # Pasar por GRU
        out, _ = self.gru(x)
        
        # Tomar la salida del último paso temporal
        out = out[:, -1, :]
        out = self.dropout(out)
        out = self.fc(out)
        
        return out

# ### 5. Optimización de Hiperparámetros con Optuna

def objective(trial):
    """
    Función objetivo para Optuna
    """
    # Espacio de búsqueda de hiperparámetros
    window_size = trial.suggest_int('window_size', 8, 48)
    hidden_size = trial.suggest_categorical('hidden_size', [16, 32, 64, 128])
    num_layers = trial.suggest_int('num_layers', 1, 3)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    
    # Crear secuencias
    categorical_cols_encoded = [col + '_encoded' for col in categorical_cols]
    
    X_train_num, X_train_cat, y_train, _ = create_sequences(
        train_scaled, window_size, target_col, numerical_cols, categorical_cols_encoded
    )
    X_val_num, X_val_cat, y_val, _ = create_sequences(
        val_scaled, window_size, target_col, numerical_cols, categorical_cols_encoded
    )
    
    # Crear datasets y dataloaders
    train_dataset = DengueDataset(X_train_num, X_train_cat, y_train)
    val_dataset = DengueDataset(X_val_num, X_val_cat, y_val)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Configurar modelo
    num_features = len(numerical_cols)
    embedding_dims = [
        (len(label_encoders['id_bar'].classes_), min(50, len(label_encoders['id_bar'].classes_)//2)),
        (len(label_encoders['ESTRATO'].classes_), min(10, len(label_encoders['ESTRATO'].classes_)//2))
    ]
    
    model = GRUModel(num_features, embedding_dims, hidden_size, num_layers, dropout_rate).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Entrenamiento
    n_epochs = 200
    best_val_loss = float('inf')
    patience = 50
    patience_counter = 0
    
    for epoch in range(n_epochs):
        # Training
        model.train()
        train_loss = 0
        for x_num, x_cat, y in train_loader:
            x_num, x_cat, y = x_num.to(device), x_cat.to(device), y.to(device)
            
            optimizer.zero_grad()
            output = model(x_num, x_cat)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x_num, x_cat, y in val_loader:
                x_num, x_cat, y = x_num.to(device), x_cat.to(device), y.to(device)
                output = model(x_num, x_cat)
                loss = criterion(output, y)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            break
    
    return best_val_loss

# Ejecutar optimización con Optuna
print("\n### Iniciando optimización de hiperparámetros con Optuna ###")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200, show_progress_bar=True)

print("\nMejores hiperparámetros encontrados:")
print(study.best_params)
print(f"\nMejor MSE en validación: {study.best_value:.4f}")

# ### 6. Entrenamiento del Modelo Final

# Obtener mejores hiperparámetros
best_params = study.best_params
window_size = best_params['window_size']
hidden_size = best_params['hidden_size']
num_layers = best_params['num_layers']
dropout_rate = best_params['dropout_rate']
learning_rate = best_params['learning_rate']
batch_size = best_params['batch_size']

print("\n### Entrenando modelo final con datos combinados ###")

# Combinar datos de entrenamiento y validación
combined_data = pd.concat([train_data, val_data])
combined_scaled = combined_data.copy()
combined_scaled[numerical_cols] = scaler.transform(combined_data[numerical_cols])

# Crear secuencias con datos combinados
categorical_cols_encoded = [col + '_encoded' for col in categorical_cols]
X_combined_num, X_combined_cat, y_combined, _ = create_sequences(
    combined_scaled, window_size, target_col, numerical_cols, categorical_cols_encoded
)

# Crear dataset y dataloader
combined_dataset = DengueDataset(X_combined_num, X_combined_cat, y_combined)
combined_loader = DataLoader(combined_dataset, batch_size=batch_size, shuffle=True)

# Configurar modelo final
num_features = len(numerical_cols)
embedding_dims = [
    (len(label_encoders['id_bar'].classes_), min(50, len(label_encoders['id_bar'].classes_)//2)),
    (len(label_encoders['ESTRATO'].classes_), min(10, len(label_encoders['ESTRATO'].classes_)//2))
]

model_final = GRUModel(num_features, embedding_dims, hidden_size, num_layers, dropout_rate).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_final.parameters(), lr=learning_rate)

# Entrenamiento del modelo final
n_epochs = 200
train_losses = []

for epoch in range(n_epochs):
    model_final.train()
    epoch_loss = 0
    
    for x_num, x_cat, y in combined_loader:
        x_num, x_cat, y = x_num.to(device), x_cat.to(device), y.to(device)
        
        optimizer.zero_grad()
        output = model_final(x_num, x_cat)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(combined_loader)
    train_losses.append(avg_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f"Época {epoch+1}/{n_epochs}, Pérdida: {avg_loss:.4f}")

# Visualizar curva de pérdida
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, n_epochs+1)), y=train_losses,
                        mode='lines', name='Pérdida de Entrenamiento'))
fig.update_layout(title='Curva de Pérdida durante el Entrenamiento',
                  xaxis_title='Época', yaxis_title='MSE')
fig.show()

# ### 7. Generación de Pronósticos para 2022

print("\n### Generando pronósticos para 2022 ###")

# Preparar datos para pronóstico
all_data = df.copy()
all_data_scaled = all_data.copy()
all_data_scaled[numerical_cols] = scaler.transform(all_data[numerical_cols])

# Función para generar pronósticos autorregresivos
def generate_forecasts(model, data, window_size, n_weeks_ahead, year):
    """
    Genera pronósticos autorregresivos para un año completo
    """
    model.eval()
    forecasts = []
    
    # Para cada barrio
    for barrio in data['id_bar'].unique():
        barrio_data = data[data['id_bar'] == barrio].sort_index()
        barrio_id = barrio_data['id_bar'].iloc[0]
        estrato_encoded = barrio_data['ESTRATO_encoded'].iloc[0]
        
        # Obtener última secuencia conocida
        last_sequence_num = barrio_data[numerical_cols].iloc[-window_size:].values
        last_sequence_cat = np.array([[barrio_data['id_bar_encoded'].iloc[0], 
                                      estrato_encoded] for _ in range(window_size)])
        
        # Generar pronósticos semana a semana
        for week in range(1, n_weeks_ahead + 1):
            # Preparar entrada
            x_num = torch.FloatTensor(last_sequence_num).unsqueeze(0).to(device)
            x_cat = torch.LongTensor(last_sequence_cat).unsqueeze(0).to(device)
            
            # Hacer predicción
            with torch.no_grad():
                pred_scaled = model(x_num, x_cat).cpu().numpy()[0, 0]
            
            # Desescalar predicción (solo para la columna de dengue)
            dengue_idx = numerical_cols.index('dengue')
            pred_original_scale = pred_scaled * scaler.scale_[dengue_idx] + scaler.mean_[dengue_idx]
            
            # Asegurar que la predicción no sea negativa
            pred_original_scale = max(0, pred_original_scale)
            
            # Guardar predicción
            forecasts.append({
                'id_bar': barrio_id,
                'anio': year,
                'semana': week,
                'dengue': pred_original_scale
            })
            
            # Actualizar secuencia para próxima predicción
            # Crear nueva fila con la predicción
            new_row = last_sequence_num[-1].copy()
            new_row[dengue_idx] = pred_scaled
            
            # Actualizar ventana deslizante
            last_sequence_num = np.vstack([last_sequence_num[1:], new_row])
    
    return pd.DataFrame(forecasts)

# Generar pronósticos para 2022
forecasts_2022 = generate_forecasts(model_final, all_data_scaled, window_size, 52, 2022)

print(f"\nPronósticos generados: {len(forecasts_2022)} registros")
print("\nEstadísticas de pronósticos:")
print(forecasts_2022['dengue'].describe())

# ### 8. Creación del Archivo de Salida

# Crear DataFrame con formato requerido
output_df = pd.DataFrame()
output_df['id'] = (forecasts_2022['id_bar'].astype(str) + '_' + 
                   forecasts_2022['anio'].astype(str) + '_' + 
                   forecasts_2022['semana'].apply(lambda x: f"{x:02d}"))
output_df['dengue'] = forecasts_2022['dengue'].round(2)

# Verificar formato
print("\nPrimeras filas del archivo de salida:")
print(output_df.head(10))
print(f"\nTotal de registros: {len(output_df)}")

# Guardar archivo CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f'pronosticos_GRU_C_{timestamp}.csv'
output_df.to_csv(output_filename, index=False)
print("\nArchivo 'pronosticos_dengue_2022.csv' guardado exitosamente.")

# Visualización de algunos pronósticos
sample_barrios = forecasts_2022['id_bar'].unique()[:5]
fig = go.Figure()

for barrio in sample_barrios:
    barrio_forecast = forecasts_2022[forecasts_2022['id_bar'] == barrio]
    fig.add_trace(go.Scatter(
        x=barrio_forecast['semana'],
        y=barrio_forecast['dengue'],
        mode='lines+markers',
        name=f'Barrio {barrio}'
    ))

fig.update_layout(
    title='Pronósticos de Dengue para 2022 (Muestra de 5 Barrios)',
    xaxis_title='Semana del Año',
    yaxis_title='Casos de Dengue',
    hovermode='x unified'
)
fig.show()

print("\n### Proceso completado exitosamente ###")

[I 2025-06-23 12:53:55,342] A new study created in memory with name: no-name-5adf89b7-dba1-4ba5-9cf8-1c43f0185652


Dispositivo de cómputo: cuda
Dimensiones del DataFrame: (3680, 20)

Primeras filas:
          id  id_bar  anio  semana  ESTRATO  area_barrio  dengue  \
0  4_2015_01       4  2015       1      3.0        0.560     0.0   
1  5_2015_01       5  2015       1      3.0        0.842     0.0   
2  3_2015_01       3  2015       1      1.0        0.781     0.0   
3  8_2015_01       8  2015       1      2.0        0.394     0.0   
4  9_2015_01       9  2015       1      2.0        0.292     0.0   

   concentraciones  vivienda  equipesado  sumideros  maquina  lluvia_mean  \
0              0.0       0.0         0.0        0.0      0.0     0.000651   
1              0.0       0.0         0.0        0.0      0.0     0.000651   
2              0.0       0.0         0.0        0.0      0.0     0.000651   
3              0.0       0.0         0.0        0.0      0.0     0.000651   
4              0.0       0.0         0.0        0.0      0.0     0.000651   

   lluvia_var  lluvia_max  lluvia_min  tempe

  0%|          | 0/200 [00:00<?, ?it/s]

[I 2025-06-23 12:54:03,082] Trial 0 finished with value: 0.09404972195625305 and parameters: {'window_size': 28, 'hidden_size': 64, 'num_layers': 2, 'dropout_rate': 0.11603062481544174, 'learning_rate': 0.006565545714341804, 'batch_size': 64}. Best is trial 0 with value: 0.09404972195625305.
[I 2025-06-23 12:54:15,137] Trial 1 finished with value: 0.06795666553080082 and parameters: {'window_size': 44, 'hidden_size': 32, 'num_layers': 3, 'dropout_rate': 0.10824281349224583, 'learning_rate': 0.0013764512306795925, 'batch_size': 32}. Best is trial 1 with value: 0.06795666553080082.
[I 2025-06-23 12:54:20,711] Trial 2 finished with value: 0.1028464784224828 and parameters: {'window_size': 25, 'hidden_size': 32, 'num_layers': 2, 'dropout_rate': 0.22050096586937054, 'learning_rate': 0.007639010642902058, 'batch_size': 128}. Best is trial 1 with value: 0.06795666553080082.
[I 2025-06-23 12:54:47,114] Trial 3 finished with value: 0.10960474237799644 and parameters: {'window_size': 15, 'hidden


### Generando pronósticos para 2022 ###

Pronósticos generados: 520 registros

Estadísticas de pronósticos:
count    520.000000
mean       1.386400
std        0.542368
min        0.000000
25%        0.960718
50%        1.358620
75%        1.680529
max        3.910512
Name: dengue, dtype: float64

Primeras filas del archivo de salida:
          id  dengue
0  4_2022_01    0.39
1  4_2022_02    0.67
2  4_2022_03    1.20
3  4_2022_04    2.07
4  4_2022_05    2.64
5  4_2022_06    2.57
6  4_2022_07    2.23
7  4_2022_08    1.97
8  4_2022_09    1.99
9  4_2022_10    1.96

Total de registros: 520

Archivo 'pronosticos_dengue_2022.csv' guardado exitosamente.



### Proceso completado exitosamente ###
